In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.linalg import lstsq
import sys
import os
function_essential_path = r"H:\Desktop\Environement_Trading_Developement\FunctionEssential"
sys.path.append(function_essential_path)
import utils as us


In [1]:

def compute_TPL(prices, num_points=20):
    """
    Calcule la Trigonometric Price Line (TPL) en utilisant l'interpolation trigonométrique 
    en résolvant le système des moindres carrés comme dans le papier.

    :param prices: Liste des prix historiques (numpy array 1D).
    :param num_points: Nombre de fréquences utilisées (convergence points).
    :return: Tableau TPL interpolé.
    """
    N = len(prices)  # Nombre de points dans la série temporelle
    t = np.linspace(0, 1, N)  # Normalisation du temps entre [0,1]

    # Étape 1 : Construction de la base trigonométrique (matrice de Vandermonde)
    X = np.ones((N, 2 * num_points + 1))  # Colonne de biais (terme constant)
    for k in range(1, num_points + 1):
        X[:, 2*k-1] = np.cos(2 * np.pi * k * t)  # Cosinus
        X[:, 2*k] = np.sin(2 * np.pi * k * t)    # Sinus

    # Étape 2 : Résolution du système des moindres carrés
    coeffs, _, _, _ = lstsq(X, prices)  # Trouve les coefficients optimaux

    # Étape 3 : Reconstruction de la TPL avec les coefficients obtenus
    TPL = X @ coeffs  # Produit matriciel pour générer la courbe interpolée

    return TPL


In [2]:
data = us.get_data_forex(['EURUSD'],'H4','2005-01-01','2006-01-01')
close = data['EURUSD']['close']

NameError: name 'us' is not defined

In [ ]:
import PCY_PPL as pp

pcy = pp.compute_pcy(close,10,20,50,0.33)
ppl = pp.compute_ppl(pcy,close,10)

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.1)

fig.add_trace(go.Scatter(
    x=list(range(len(close))),
    y=close,
    mode='lines',
    name='close'
),
row=1,
col=1
)

fig.add_trace(go.Scatter(
    x=list(range(len(close))),
    y=ppl,
    mode='lines',
    name='ppl'
),
row=1,
col=1
)

fig.add_trace(go.Scatter(
    x=list(range(len(close))),
    y=pcy,
    mode='lines',
    name='pcy'
),
row=2,
col=1
)

fig.update_layout(height=600, width=600, title_text="PPL et PCY")
fig.show()

In [ ]:
min_diff = close.rolling(window=10).min()
max_diff = close.rolling(window=10).max()

In [ ]:
ppl_nan_replace = ppl.fillna(0)
TPL = compute_TPL(ppl_nan_replace)
TPL = pd.Series(TPL,index=close.index)
import plotly.graph_objects as go
import plotly.io as pio  
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=close.index,
    y=close,
    mode='lines',
    name="Close"
))

fig.add_trace(go.Scatter(
    x=TPL.index,
    y=TPL,
    mode='lines',
    name="TPL"
))

fig.update_layout(
    title="Close et TPL",
    xaxis_title="Index Numérique",
    yaxis_title="Prix/TPL",
    template="plotly_dark",
    xaxis_rangeslider_visible=False,
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    )
)
fig.show()